In [1]:
import numpy as np
import pandas as pd
import random
import cvxopt
from cvxopt import matrix, solvers
from cvxopt.modeling import variable
from tqdm.notebook import tqdm
from sklearn.svm import SVC

In [11]:
N_FOLDS = 10 # k-Fold cross validation
NUM_FEATURES = 25 # <=25 for the given data
C1,C2 = 1,8 # Classes (0-9)
KERNEL = 'RBF' # 'LINEAR', 'POLY', 'RBF'
GAMMA = 0.001 # Gamma for RBF Kernel
C = 3.0 # C for L1 regularization
POWER = 2 # Power in Polynomial kernel
COEFF = 4 # Coefficient in Polynomial kernel

In [3]:
def getData(C1,C2):
    df = pd.read_csv('2019EE10577.csv',header=None)
    df = df[(df[25] == C1) | (df[25] == C2)]
    dataset = []
    for i,row in df.iterrows():
        xi = []
        for j in range(0,NUM_FEATURES):
            xi.append(row[j])
        if(row[25]==C1):
            xi.append(-1.0)
        else:
            xi.append(1.0)
        dataset.append(xi)
    return dataset

In [4]:
def cross_validation_split(dataset):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / N_FOLDS)
    for i in range(N_FOLDS):
        fold = list()
        while len(fold) < fold_size:
            index = random.randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [5]:
dataset = getData(C1,C2)

In [6]:
dataset_split = cross_validation_split(dataset)

In [7]:
class SVM:
    def __init__(self,kernel='LINEAR',C = 3.0,gamma = 0.001, power = 2, coeff = 4):
        self.kernel = kernel
        self.C = C
        self.gamma = gamma
        self.power = power
        self.coeff = coeff
    
    def fit(self,X,y):
        X = np.array(X)
        y = np.array(y)
        num_samples,num_features = X.shape
        K = np.zeros((num_samples,num_samples))
        
        for i in (range(num_samples)):
            for j in range(num_samples):
                if self.kernel == 'LINEAR':
                    K[i][j] = np.dot(X[i],np.transpose(X[j]))
                elif self.kernel == 'POLY':
                    K[i][j] = (np.dot(X[i],np.transpose(X[j])) + self.coeff) ** self.power
                elif self.kernel == 'RBF':
                    K[i][j] = np.exp(-1 * self.gamma*np.sum(np.square(X[i]-X[j])))
        
        self.K = K
        
        H = np.zeros((num_samples,num_samples))
        for i in (range(num_samples)):
            for j in range(num_samples):
                H[i][j] = y[i]*y[j] * K[i][j]
        P = matrix(H)
        q = matrix(np.ones(num_samples) * -1)
        G = matrix(np.vstack(((np.identity(num_samples) * -1),np.identity(num_samples))))
        h = matrix(np.hstack((np.zeros(num_samples),np.ones(num_samples)*self.C)))
        A = matrix(y,(1,num_samples))
        b = matrix(0.0)
        
        solvers.options['show_progress'] = False
        soln = solvers.qp(P,q,G,h,A,b)
        alpha = np.array(soln['x']) 
        
        self.sup_idx = np.where(alpha>1e-5)[0]
        self.ind =  np.arange(len(alpha))[self.sup_idx]
        
        self.sup_x = X[self.sup_idx,:]
        self.sup_y = y[self.sup_idx]
        self.alpha = alpha[self.sup_idx]
        self.b = self.sup_y[0]
        
        for i in range(len(self.alpha)):
            if self.kernel == 'LINEAR':
                temp = np.dot(self.sup_x[i],np.transpose(self.sup_x[0]))
            elif self.kernel == 'POLY':
                temp = (np.dot(self.sup_x[i],np.transpose(self.sup_x[0]))+self.coeff)**self.power
            elif self.kernel == 'RBF':
                temp = np.exp(-1 * self.gamma*np.sum(np.square(self.sup_x[i]-self.sup_x[0])))
            self.b -= self.alpha[i] * self.sup_y[i] * temp
        
    def predict(self,X):
        preds = []
        for x in X:
            pred = 0
            for i in range(len(self.alpha)):
                if self.kernel == 'LINEAR':
                    temp = np.dot(self.sup_x[i],np.transpose(x))
                elif self.kernel == 'POLY':
                    temp = (np.dot(self.sup_x[i],np.transpose(x)) + self.coeff) ** self.power
                elif self.kernel == 'RBF':
                    temp = np.exp(-1 * self.gamma *np.sum(np.square(self.sup_x[i]-x)))
                pred += self.alpha[i] * self.sup_y[i] * temp
            pred += self.b
            if pred>=0:
                preds.append(1.0)
            else:
                preds.append(-1.0)
        return np.array(preds)

In [8]:
def getAccuracy(preds,testY):
    correct = 0
    for i in range(len(testY)):
        if preds[i] == testY[i]:
            correct +=1
    return correct/len(testY) * 100

In [9]:
def evaluate(dataset_split):
    acc_list = []
    for fold in tqdm(dataset_split):
        trainDS = list(dataset_split)
        trainDS.remove(fold)
        trainDS = sum(trainDS,[])
        testDS = list()
        
        for row in fold:
            rowCopy = list(row)
            testDS.append(rowCopy)
        
        trainX = [train[:-1] for train in trainDS]
        trainY = [train[-1] for train in trainDS]
        
        testX = [test[:-1] for test in testDS]
        testY = [test[-1] for test in testDS]
        
        cf = SVM(kernel=KERNEL,gamma=GAMMA,C=C,power=POWER,coeff=COEFF)
        cf.fit(trainX,trainY)
        
        preds = cf.predict(testX)
        
        acc = getAccuracy(preds,testY)
        acc_list.append(acc)
        
    return acc_list,np.mean(acc_list)

In [10]:
evaluate(dataset_split)

([91.93548387096774,
  96.7741935483871,
  93.54838709677419,
  96.7741935483871,
  98.38709677419355,
  88.70967741935483,
  95.16129032258065,
  95.16129032258065,
  100.0,
  98.38709677419355],
 95.48387096774194)